pca_bert_100features

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/bert_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,...,-0.368736,0.161809,-0.176456,0.019982,-0.068931,0.117525,0.303489,-0.251144,0.580060,0.538873
1,1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,...,-0.454385,-0.043330,-1.040999,-0.095080,-0.497205,-0.132479,0.267727,-0.089319,0.100436,-0.062043
2,2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,...,-0.955023,1.327444,-1.382465,0.626360,0.082504,-0.240717,0.799674,-0.157947,0.282090,0.106546
3,3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,...,0.692133,0.658792,0.368164,0.689335,-0.200350,-0.924818,-0.257367,-0.371434,0.126181,-0.100700
4,4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,...,-0.015543,-0.921747,0.575937,-0.974936,-0.201186,0.272522,-0.838430,-0.132166,-0.262502,0.632246


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,0.203139,...,-0.368736,0.161809,-0.176456,0.019982,-0.068931,0.117525,0.303489,-0.251144,0.580060,0.538873
1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,1.341340,...,-0.454385,-0.043330,-1.040999,-0.095080,-0.497205,-0.132479,0.267727,-0.089319,0.100436,-0.062043
2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,-2.739575,...,-0.955023,1.327444,-1.382465,0.626360,0.082504,-0.240717,0.799674,-0.157947,0.282090,0.106546
3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,-2.567815,...,0.692133,0.658792,0.368164,0.689335,-0.200350,-0.924818,-0.257367,-0.371434,0.126181,-0.100700
4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,1.247958,...,-0.015543,-0.921747,0.575937,-0.974936,-0.201186,0.272522,-0.838430,-0.132166,-0.262502,0.632246


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3496213959427514


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train) 

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.438525,0.438525,0.636368,0.389012,0.278626,0.278626,0.339138,0.232764


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2222    0.1019    0.1397       157
           2     0.4614    0.8312    0.5934       237
           3     0.2000    0.0385    0.0645       130

    accuracy                         0.4160       524
   macro avg     0.2945    0.3239    0.2659       524
weighted avg     0.3249    0.4160    0.3263       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.438525,0.438525,0.636368,0.389012,0.278626,0.278626,0.339138,0.232764
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.416031,0.416031,0.324868,0.326251


**AdaBoost**

In [27]:
clf_adaboost =  AdaBoostClassifier()

In [28]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [29]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5962    0.4377    0.5048       361
           2     0.5697    0.7541    0.6490       553
           3     0.5605    0.4085    0.4726       306

    accuracy                         0.5738      1220
   macro avg     0.5755    0.5334    0.5421      1220
weighted avg     0.5752    0.5738    0.5621      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3058    0.2357    0.2662       157
           2     0.4403    0.5443    0.4868       237
           3     0.2000    0.1692    0.1833       130

    accuracy                         0.3588       524
   macro avg     0.3154    0.3164    0.3121       524
weighted avg     0.3404    0.3588    0.3454       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.539344,0.539344,0.637303,0.473640,0.440840,0.440840,0.364831,0.366479
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.330748,0.331963
2,Adaboost,0.573770,0.573770,0.575239,0.562094,0.358779,0.358779,0.340368,0.345409


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9799    0.9474    0.9634       361
           2     0.9305    0.9928    0.9606       553
           3     0.9964    0.9150    0.9540       306

    accuracy                         0.9598      1220
   macro avg     0.9690    0.9517    0.9593      1220
weighted avg     0.9617    0.9598    0.9598      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3077    0.2293    0.2628       157
           2     0.4500    0.6456    0.5303       237
           3     0.2388    0.1231    0.1624       130

    accuracy                         0.3912       524
   macro avg     0.3322    0.3326    0.3185       524
weighted avg     0.3550    0.3912    0.3589       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.539344,0.539344,0.637303,0.473640,0.440840,0.440840,0.364831,0.366479
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.330748,0.331963
2,Adaboost,0.573770,0.573770,0.575239,0.562094,0.358779,0.358779,0.340368,0.345409
3,GBM,0.959836,0.959836,0.961673,0.959782,0.391221,0.391221,0.354967,0.358894


**SVM Model**

In [31]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [32]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [33]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [34]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4567    0.5983    0.5180       361
           2     0.6630    0.4412    0.5299       553
           3     0.4142    0.5131    0.4584       306

    accuracy                         0.5057      1220
   macro avg     0.5113    0.5175    0.5021      1220
weighted avg     0.5396    0.5057    0.5084      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3259    0.4650    0.3832       157
           2     0.4848    0.2700    0.3469       237
           3     0.2500    0.3231    0.2819       130

    accuracy                         0.3416       524
   macro avg     0.3536    0.3527    0.3373       524
weighted avg     0.3790    0.3416    0.3416       524



In [35]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.438525,0.438525,0.636368,0.389012,0.278626,0.278626,0.339138,0.232764
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.416031,0.416031,0.324868,0.326251
2,SVM,0.505738,0.505738,0.539571,0.508421,0.341603,0.341603,0.378959,0.341638


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5160    0.5346    0.5252       361
           2     0.5805    0.7306    0.6469       553
           3     0.4667    0.2288    0.3070       306

    accuracy                         0.5467      1220
   macro avg     0.5211    0.4980    0.4930      1220
weighted avg     0.5329    0.5467    0.5256      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2667    0.2803    0.2733       157
           2     0.4476    0.5401    0.4895       237
           3     0.1918    0.1077    0.1379       130

    accuracy                         0.3550       524
   macro avg     0.3020    0.3093    0.3002       524
weighted avg     0.3299    0.3550    0.3375       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.539344,0.539344,0.637303,0.473640,0.440840,0.440840,0.364831,0.366479
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.330748,0.331963
2,Adaboost,0.573770,0.573770,0.575239,0.562094,0.358779,0.358779,0.340368,0.345409
3,GBM,0.959836,0.959836,0.961673,0.959782,0.391221,0.391221,0.354967,0.358894
4,SVM,0.526230,0.526230,0.529530,0.485342,0.391221,0.391221,0.345633,0.348132
5,KNN,0.546721,0.546721,0.532857,0.525639,0.354962,0.354962,0.329901,0.337491


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4182    0.4958    0.4537       361
           2     0.5496    0.6112    0.5788       553
           3     0.3672    0.2124    0.2692       306

    accuracy                         0.4770      1220
   macro avg     0.4450    0.4398    0.4339      1220
weighted avg     0.4650    0.4770    0.4641      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3397    0.4522    0.3880       157
           2     0.4958    0.4979    0.4968       237
           3     0.1948    0.1154    0.1449       130

    accuracy                         0.3893       524
   macro avg     0.3434    0.3552    0.3432       524
weighted avg     0.3744    0.3893    0.3769       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.539344,0.539344,0.637303,0.473640,0.440840,0.440840,0.364831,0.366479
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.330748,0.331963
2,Adaboost,0.573770,0.573770,0.575239,0.562094,0.358779,0.358779,0.340368,0.345409
3,GBM,0.959836,0.959836,0.961673,0.959782,0.391221,0.391221,0.354967,0.358894
4,SVM,0.526230,0.526230,0.529530,0.485342,0.391221,0.391221,0.345633,0.348132
5,KNN,0.546721,0.546721,0.532857,0.525639,0.354962,0.354962,0.329901,0.337491
6,GNB,0.477049,0.477049,0.464981,0.464113,0.389313,0.389313,0.374358,0.376917
